In [30]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

import numpy as np
import plotly.graph_objects as go
from evo.tools import file_interface

from gorbslam.nn.orbslam_processor import ORBSLAMResults, ORBSLAMProcessor
from gorbslam.common.plotting_utils import create_scattermapbox, create_map_fig
from os import path

# TRAJECTORY_NAME = 'vabadussild'
# TRAJECTORY_NAME = 'vabadusepst'
TRAJECTORY_NAME = 'oldtown'

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME))


AttributeError: 'ZMQInteractiveShell' object has no attribute 'extensions'

In [ ]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()

In [ ]:
processor.fit_trajectories()


30/30 [==============================] - 0s 682us/step


In [ ]:
# processor.model.best_hps.values

In [ ]:
processor.model.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 207)               828       
                                                                 
 dense_5 (Dense)             (None, 994)               206752    
                                                                 
 dense_6 (Dense)             (None, 40)                39800     
                                                                 
 dense_7 (Dense)             (None, 25)                1025      
                                                                 
 dense_8 (Dense)             (None, 279)               7254      
                                                                 
 dense_9 (Dense)             (None, 3)                 840       
                                                                 
Total params: 256,499
Trainable params: 256,499
Non-tr

In [ ]:
processor.save_trajectories()

In [ ]:
fig_2d = processor.create_2d_plot_utm()
fig_2d.show()

In [ ]:
fig_slam = processor.create_2d_plot_slam()
fig_slam.show()

In [ ]:
fig_map = processor.create_map_plot()
fig_map.show()

In [ ]:

from utils import create_training_splits


training_data = (processor.orbslam.mapping.trajectory, processor.orbslam.mapping.trajectory_gt_utm)
extra_data = (processor.orbslam.localization[0].trajectory, processor.orbslam.localization[0].trajectory_gt_utm)

training, validation, testing = create_training_splits(training_data, extra_data, 0.20)


ModuleNotFoundError: No module named 'utils'

In [ ]:
fig_training = go.Figure()

fig_training.add_trace(go.Scatter(
    x=training[0][:, 0],
    y=training[0][:, 2],
    mode='markers',
    name="training"
))
fig_training.add_trace(go.Scatter(
    x=validation[0][:, 0],
    y=validation[0][:, 2],
    mode='markers',
    name="validation"
))
fig_training.add_trace(go.Scatter(
    x=testing[0][:, 0],
    y=testing[0][:, 2],
    mode='markers',
    name="testing"
))

fig_training.update_layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=1200
)

print(f'training: {training[0].shape}')
print(f'validation: {validation[0].shape}')
print(f'testing: {testing[0].shape}')

fig_training.show()